In [234]:
#
# classifer notebook
#
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Lambda, Compose
from LandmarkDataset import LandmarkDataset

#
# MODEL
#

class NN(torch.nn.Module):
    
    def __init__(self, input_size, hidden_size, num_classes):
        super(NN, self).__init__()
        self.fc1 = torch.nn.Linear(input_size, hidden_size)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

    
#
# PARAMS
#

batch_size = 4

input_size = 2 * (21 * 3) + 12 #138
hidden_size = 96
num_classes = 3

learning_rate = .01
num_epochs = 100


#
# INITIAL DATA
#

transformations = Compose([
    Lambda(lambda x: torch.tensor(x.values).float())
])

dataset = LandmarkDataset("/home/jovyan/train/data/landmarks_indexed.csv",
                              transform=transformations)

dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

#
# INIT MODELS, LOSS FN, GRAD
#

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NN(input_size, hidden_size, num_classes).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

#
# TRAIN
#

running_loss_epoch = 0
for epoch in range(num_epochs):
    for batch_idx, (labels, landmarks) in enumerate(dataloader):  

        # zero out accumulated gradients
        optimizer.zero_grad()
        
        # forward pass
        outputs = model(landmarks)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        loss.backward()
        optimizer.step()
        
        # NB: len(dataloader) is num of batches
        running_loss_epoch += loss.item()
        if ((epoch+1) % 10 == 0) and ((batch_idx+1) % len(dataloader) == 0):
            print ('Epoch [{}/{}], Loss: {:.4f}, Running Loss (per N epoch): {:.4f}' 
                   .format(epoch+1, num_epochs, loss.item(), running_loss_epoch))
            running_loss_epoch = 0


Epoch [10/100], Loss: 1.1092, Running Loss (per N epoch): 81.9835
Epoch [20/100], Loss: 1.1384, Running Loss (per N epoch): 75.9194
Epoch [30/100], Loss: 1.0301, Running Loss (per N epoch): 72.5314
Epoch [40/100], Loss: 0.8820, Running Loss (per N epoch): 67.8351
Epoch [50/100], Loss: 0.9010, Running Loss (per N epoch): 61.5773
Epoch [60/100], Loss: 0.1074, Running Loss (per N epoch): 53.6545
Epoch [70/100], Loss: 0.2210, Running Loss (per N epoch): 50.7069
Epoch [80/100], Loss: 0.8235, Running Loss (per N epoch): 48.0273
Epoch [90/100], Loss: 0.0251, Running Loss (per N epoch): 47.8699
Epoch [100/100], Loss: 1.1275, Running Loss (per N epoch): 50.6602
Done


In [235]:
#
# EVAL
#

accuracy = 0
count = 0
with torch.no_grad():
    
    for batch_idx, (labels, landmarks) in enumerate(dataloader):  
        out = model(landmarks)
        _, klass = torch.max(out.data, 1)
        print(klass, labels, klass==labels)
        accuracy += (klass == labels).sum().item()
        count += len(labels)
        

print('--------')
print("Accuracy {}/{} : {:.4f}".format(accuracy, count, accuracy/count))


tensor([0, 1, 1, 1]) tensor([1, 1, 1, 1]) tensor([False,  True,  True,  True])
tensor([0, 2, 1, 1]) tensor([0, 2, 1, 1]) tensor([True, True, True, True])
tensor([1, 0, 0, 2]) tensor([1, 0, 0, 2]) tensor([True, True, True, True])
tensor([1, 2, 1, 1]) tensor([2, 2, 2, 1]) tensor([False,  True, False,  True])
tensor([1, 0, 2, 0]) tensor([0, 0, 2, 0]) tensor([False,  True,  True,  True])
tensor([1, 1, 2, 1]) tensor([1, 1, 2, 2]) tensor([ True,  True,  True, False])
tensor([0]) tensor([0]) tensor([True])
--------
Accuracy 20/25 : 0.8000


In [ ]:
print("Done")